In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


In [2]:
mutations_df = pd.read_csv('../data/WES_variants.csv')
mutations_df.head()

,SAMPLE,COSMIC_ID,Cancer Type,Gene,Transcript,cDNA,AA,Classification,Gene_list,Recurrence Filter,Subs,Truncating,inframe
0,KARPAS-45,907272,ALL,A1BG,ENST00000263100,c.842A>G,p.Y281C,missense,NaN,NaN,NaN,NaN,NaN
1,Jurkat,998184,ALL,A1BG,ENST00000263100,c.589G>A,p.A197T,missense,NaN,NaN,NaN,NaN,NaN
2,KARPAS-45,907272,ALL,A1BG,ENST00000263100,c.571G>A,p.A191T,missense,NaN,NaN,2.0,NaN,NaN
3,HT-115,907289,COAD/READ,A1BG,ENST00000263100,c.1456G>A,p.D486N,missense,NaN,NaN,NaN,NaN,NaN
4,KM12,905989,COAD/READ,A1BG,ENST00000263100,c.770C>A,p.T257N,missense,NaN,NaN,NaN,NaN,NaN


In [ ]:
#mutations_df.count()

grouped_mut_df = mutations_df.groupby('COSMIC_ID')

grouped_mut_df.count()



In [2]:
dose_response_df = pd.read_csv('../data/v17.3_fitted_dose_response.csv', index_col='IC50_RESULTS_ID')
dose_response_df.head()

,DATASET_VERSION,COSMIC_ID,CELL_LINE_NAME,DRUG_ID,DRUG_NAME,PUTATIVE_TARGET,MAX_CONC_MICROMOLAR,MIN_CONC_MICROMOLAR,LN_IC50,AUC,RMSE,Z_SCORE
IC50_RESULTS_ID,,,,,,,,,,,,
1,17.3,683665,MC-CAR,1,Erlotinib,EGFR,2.0,0.007812,2.453524,0.982610,0.021678,-0.015505
1482,17.3,684055,ES3,1,Erlotinib,EGFR,2.0,0.007812,3.376592,0.985169,0.029915,0.779999
1791,17.3,684057,ES5,1,Erlotinib,EGFR,2.0,0.007812,3.614664,0.983207,0.031201,0.985170
2177,17.3,684059,ES7,1,Erlotinib,EGFR,2.0,0.007812,3.223394,0.984574,0.093857,0.647971
2754,17.3,684062,EW-11,1,Erlotinib,EGFR,2.0,0.007812,2.486405,0.946034,0.087280,0.012832


In [3]:
dose_response_trimmed_df = dose_response_df[['COSMIC_ID', 'DRUG_ID', 'LN_IC50']]

In [4]:
wes_scored = pd.read_csv("../data/wes_scored_transposed.tsv",sep = "\t")
wes_scored.set_index("Unnamed: 0",drop= True ,inplace=True)
wes_scored.index.names = ['COSMIC_ID']
wes_scored.head()

,1,2,9,10,12,13,14,15,16,18,...,101060321,101927546,101927722,101928638,102724473,102724928,105375355,105378803,107403068,109731405
COSMIC_ID,,,,,,,,,,,,,,,,,,,,,
907268,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
907269,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
907270,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
907271,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
907272,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
joined_df = pd.merge(wes_scored, dose_response_trimmed_df, on='COSMIC_ID')

#dose_response_df.join(wes_scored.set_index('COSMIC_ID'), on='COSMIC_ID')
joined_df.head()

,COSMIC_ID,1,2,9,10,12,13,14,15,16,...,101927722,101928638,102724473,102724928,105375355,105378803,107403068,109731405,DRUG_ID,LN_IC50
0,907268,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,2.726708
1,907268,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,-4.983852
2,907268,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5,2.647692
3,907268,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6,2.919170
4,907268,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9,-0.176494


In [8]:
joined_df.to_csv("../data/wes_ic50_full.tsv",sep = "\t")

In [6]:
joined_df_efgr = joined_df[joined_df.DRUG_ID == 1]

In [7]:
joined_df_efgr.head()

,COSMIC_ID,1,2,9,10,12,13,14,15,16,...,101927722,101928638,102724473,102724928,105375355,105378803,107403068,109731405,DRUG_ID,LN_IC50
0,907268,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,2.726708
259,907269,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,3.175124
518,907270,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,3.332820
777,907271,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,2.894932
1244,907273,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,2.717920


In [8]:
joined_df_efgr.to_csv("../data/wes_ic50_EFGR.tsv",sep = "\t")

In [16]:
mutation_dose_df = joined_df[['COSMIC_ID', 'LN_IC50', 'cDNA','AA','DRUG_ID']]

In [17]:
mutation_dose_df.head()

,COSMIC_ID,LN_IC50,cDNA,AA,DRUG_ID
IC50_RESULTS_ID,,,,,
1,683665,2.453524,c.26G>A,p.R9H,1
1,683665,2.453524,c.3309+5G>T,p.?,1
1,683665,2.453524,c.1436C>T,p.T479I,1
1,683665,2.453524,c.2206G>A,p.D736N,1
1,683665,2.453524,c.2804C>G,p.T935R,1


In [18]:
mutation_dose_df.reset_index(drop=True, inplace=True)

In [21]:
mutation_dose_df.head()

,COSMIC_ID,LN_IC50,cDNA,AA,DRUG_ID
0,683665,2.453524,c.26G>A,p.R9H,1
1,683665,2.453524,c.3309+5G>T,p.?,1
2,683665,2.453524,c.1436C>T,p.T479I,1
3,683665,2.453524,c.2206G>A,p.D736N,1
4,683665,2.453524,c.2804C>G,p.T935R,1


In [20]:
mutation_dose_df.to_csv('../data/wes_dose.csv')

In [23]:
mutation_dose_df.count()

COSMIC_ID    102663661
LN_IC50      102663661
cDNA         102652824
AA           102652824
DRUG_ID      102663661
dtype: int64